In [14]:
from tensorflow.keras.models import load_model
import numpy as np
import random
from music21 import *

In [2]:
ind = np.random.randint(0,500)
model = load_model("model.h5")
random_music = [137, 6, 6,137, 6, 137, 6, 327, 103, 6, 338, 103, 249, 185, 338, 338, 338, 338, 118, 347, 338, 249, 119, 351, 351, 351, 129, 107, 23, 118, 351, 129]
random_music = np.array(random_music)
num_of_steps = 32
predictions=[]
for i in range(500):

    random_music = random_music.reshape(1,num_of_steps,1)

    prob  = model.predict(random_music)[0]
    y_pred= np.argmax(prob,axis=0)
    predictions.append(y_pred)

    random_music = np.insert(random_music[0],len(random_music[0]),y_pred)
    random_music = random_music[1:]
    
print(predictions)

[326, 185, 102, 180, 82, 6, 118, 6, 165, 129, 165, 308, 118, 326, 68, 163, 68, 113, 321, 6, 163, 326, 326, 326, 326, 129, 104, 242, 172, 341, 77, 165, 77, 165, 136, 329, 329, 327, 104, 185, 321, 321, 321, 321, 6, 125, 6, 23, 145, 83, 269, 89, 337, 82, 177, 108, 177, 148, 177, 108, 108, 332, 337, 163, 104, 329, 329, 125, 163, 321, 321, 321, 321, 125, 321, 327, 308, 308, 23, 286, 290, 290, 23, 286, 23, 92, 290, 122, 326, 290, 23, 290, 290, 23, 290, 23, 290, 23, 92, 92, 23, 118, 23, 303, 337, 337, 23, 303, 57, 231, 337, 23, 303, 57, 231, 337, 23, 135, 92, 23, 23, 337, 23, 303, 57, 231, 337, 23, 303, 57, 231, 303, 23, 303, 57, 231, 337, 23, 303, 57, 231, 337, 23, 135, 92, 23, 23, 337, 23, 303, 57, 231, 303, 303, 23, 303, 57, 231, 303, 23, 303, 57, 231, 337, 23, 135, 92, 23, 23, 337, 23, 303, 57, 231, 337, 23, 303, 57, 231, 303, 23, 303, 57, 231, 337, 23, 303, 57, 231, 337, 23, 135, 92, 23, 23, 337, 23, 303, 57, 231, 303, 303, 23, 303, 57, 231, 303, 23, 303, 57, 231, 337, 23, 135, 92, 23, 2

In [3]:
def convert_to_midi(prediction_output):
   
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                
                cn=int(current_note)
                new_note = note.Note(cn)
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
                
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
            
        # pattern is a note
        else:
            
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 1
    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

TypeError: argument of type 'numpy.int64' is not iterable

In [11]:
import pickle
with open("unique_notes.pkl","rb") as p:
    unique_notes = pickle.load(p)


In [12]:
x_int_to_note = dict((number, note_) for number, note_ in enumerate(unique_notes)) 
predicted_notes = [x_int_to_note[i] for i in predictions]

In [15]:
convert_to_midi(predicted_notes)